Here I'm using Pydantic to test the response from an LLM model. If errors are encountered, a new request to the LLM API will be sent containing error details in the prompt. 

In [ ]:
# Import necessary packages
from pydantic import BaseModel, ValidationError, Field, EmailStr
from typing import List, Literal, Optional
import json
from datetime import date
# Load environment variables from a .env file - using the dotenv library - that is where the OpenAI API key is stored
from dotenv import load_dotenv
import openai

In [8]:
# Load environment variables from a .env file
load_dotenv()
#Initlalize OpenAI API key
#openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

In [15]:
#Create a sample user input
user_input_json = '''
{
    "name": "Jan Uzytkownik",
    "email": "jan.uzytkownik@example.com",
    "query": "I forgot my password. Treat this very urgently please.",
    "order_number": null,
    "purchase_date": null
}
'''

In [16]:
# Defne the Pydantic model for validation
class UserInput(BaseModel):
    name: str
    email: EmailStr
    query: str
    order_id: Optional[int] = Field(
        None,
        description="5-digit order number (cannot start with 0)",
        ge=10000,
        le=99999
    )
    transaction_date: Optional[date] = None

In [17]:
#Create an instance of the model using the sample user input
try:
    user_input = UserInput.parse_raw(user_input_json)
    print("User input is valid:", user_input)
except ValidationError as e:
    print("Validation error:", e.json())

User input is valid: name='Jan Uzytkownik' email='jan.uzytkownik@example.com' query='I forgot my password. Treat this very urgently please.' order_id=None transaction_date=None


/var/folders/g9/l7t9y9813fjggxq0lycxgg4c0000gp/T/ipykernel_50383/1808375781.py:3: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  user_input = UserInput.parse_raw(user_input_json)


In [18]:
#Define a new CustomerQuery model that inherits from UserInput
class CustomerQuery(UserInput):
    department: str = Field(..., description="Department to route the query to")
    category: Literal['billing', 'technical', 'general'] = Field(..., description="Category of the query - billing, technical, or general")
    priority: Literal['low', 'medium', 'high'] = Field(..., description="Priority level of the query")
    tags: Optional[List[str]] = Field(..., max_items=5, description="Up to 5 keyword tags")

In [ ]:
#Create valid sample customer query data in JSON format to guide the LLM

valid_customer_query_json = '''
{
    "name": "Jan Uzytkownik",
    "email": "jan.uzytkownik@example.com",
    "query": "I forgot my password. Treat this very urgently please.",
    "order_number": 12345,
    "purchase_date": "2023-01-01",
    department: "Support",
    category: "technical",
    priority: "high",
    tags: ["password", "login", "account"]
}
'''


In [20]:
# Create prompt with user data and expected JSON structure
prompt = f"""
Please analyze this user query\n {user_input.model_dump_json(indent=2)}:

Return your analysis as a JSON object matching this exact structure 
and data types:
{valid_customer_query_json}

Respond ONLY with valid JSON. Do not include any explanations or 
other text or formatting before or after the JSON object.
"""

print(prompt)


Please analyze this user query
 {
  "name": "Jan Uzytkownik",
  "email": "jan.uzytkownik@example.com",
  "query": "I forgot my password. Treat this very urgently please.",
  "order_id": null,
  "transaction_date": null
}:

Return your analysis as a JSON object matching this exact structure 
and data types:

{
    "name": "Jan Uzytkownik",
    "email": "jan.uzytkownik@example.com",
    "query": "I forgot my password.",
    "order_number": 12345,
    "purchase_date": "2023-01-01",
    department: "Support",
    category: "technical",
    priority: "high",
    tags: ["password", "login", "account"]
}


Respond ONLY with valid JSON. Do not include any explanations or 
other text or formatting before or after the JSON object.



In [21]:
# Define a function to call the LLM
def call_llm(prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [22]:
# Get response from LLM
response_content = call_llm(prompt)
print(response_content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}